In [ ]:
# Update the key 'transform' to clean up the csv file output
# Convert into a string
profile['transform'] = str(profile['transform'])
# Replace the special characters and comma with space
profile['transform'] = profile['transform'].replace("|", " ").replace("\n", " ")
profile['transform'] = profile['transform'].replace(",", " ")
# Replace multiple spaces with a single space
profile['transform'] = ' '.join(profile['transform'].split())

In [ ]:
pattern = ("\| (?P<a>[-+]?\d*\.\d+|\d+), (?P<b>[-+]?\d*\.\d+|\d+), (?P<c>[-+]?\d*\.\d+|\d+)"
           " (?P<d>[-+]?\d*\.\d+|\d+)")
compiled = re.compile(pattern)
l = ["| 33.0, 33, 4 ", metadata_df_raw_loaded['DSM']['transform'][0]]

for elem in l:
    m = re.search(pattern, elem)
    print(m.group('a') + " " + m.group('b') + " " + m.group('c'))

In [ ]:
class Address:
    """
    A class to represent an address in Flanders
    ...
    Attributes
    ----------
    input_address : str
        Input address from the user
    valid_address_conditional: bool
        True if the input address is valid
    suggestions: dict
        Suggestions for the input address
    address_info: dict
        Detailed address information, including location
    address: str
        Formatted address
    street_name: str
        Street name
    house_no: str
        House number
    municipality: str
        Municipality name
    zipcode: str
        Municipality zipcode
    Location: dict
        Location in WGS87 and Lambert73 coordinates
    building_polygon
        Shapely polygon object representing the building
    building_coords
        Coordinates of the building polygon with n sides in the form
        [[x1, y1]
         [x2, y2]
         ...
         [xn, yn]]

    Methods
    -------
    verify_address(address: str)
        Checks if a given address is a valid address in Flanders
        Makes suggestions for incomplete addresses
    get_address_location(address: str) -> dict
        Gets the location and formatted address for an address in Flanders
    get_building_shape(address_params: dict)
        Gets the shape of a building in Flanders with a given address
    """

    def __init__(self, input_address: str):
        """
        Initializes an instance of the class Address
        :param input_address: input address from the user
        """
        # Assign attribute icon
        self.input_address = input_address

        # Verify the address
        self.valid_address_conditional, self.suggestions = self.verify_address(
            self.input_address
        )

        # For a valid address, get the location
        if self.valid_address_conditional:
            self.address_info = self.get_address_location(self.input_address)
        else:
            return None

        # Attributes for address information
        location_result = self.address_info["LocationResult"][0]
        self.address = location_result["FormattedAddress"]
        self.street_name = location_result["Thoroughfarename"]
        self.house_no = location_result["Housenumber"]
        self.municipality = location_result["Municipality"]
        self.zipcode = location_result["Zipcode"]
        self.Location = location_result["Location"]

        # Get the polygon representing the building
        address_params = {
            attr: getattr(self, attr)
            for attr in ["municipality", "zipcode", "street_name", "house_no"]
        }

        self.building_polygon, self.building_coords = self.get_building_shape(
            address_params
        )

    def __str__(self):
        """
        Prints the address
        """
        return f"{self.address}"

    @staticmethod
    def verify_address(address: str, no_suggestions=10):
        """
        Verifies whether an address is in Flanders
        Makes suggestions for incomplete addresses
        :param address: A string representing an address
        :return valid_address_conditional: A boolean that is True if the address is valid
        :return suggestions: Suggestions for addresses
        """
        # API request from Geopunt Flanders
        url = f"https://loc.geopunt.be/v4/Suggestion?q={address}&c={no_suggestions}"
        # Get the address suggestions
        try:
            suggestions = req.get(url)
            suggestions = suggestions.json()
            valid_address_conditional = len(suggestions["SuggestionResult"]) == 1
        except:
            valid_address_conditional = False  # address not found

        return valid_address_conditional, suggestions

    @staticmethod
    def get_address_location(address: str) -> dict:
        """
        Gets the location and formatted address for an address in Flanders
        :param input_address: A string representing an address
        :return address_info: Suggestions for addresses
        """
        # API request from Geopunt Flanders
        no_suggestions = 1
        url = f"https://loc.geopunt.be/v4/Location?q={address}&c={no_suggestions}"
        # Get the address suggestions
        try:
            address_info = req.get(url)
            address_info = address_info.json()
        except:
            address_info = {}

        return address_info

    @staticmethod
    def get_building_shape(address_params: dict):
        """
        Gets the shape of a building in Flanders with a given address
        :param input_address: A dictionary representing an address
        :return polygon_building: Suggestions for addresses
        """
        # URL for the address match
        url_address_match = "https://api.basisregisters.vlaanderen.be/v1/adresmatch"
        # API request for the address match
        address_match = req.get(
            url_address_match,
            params={
                "gemeentenaam": address_params["municipality"],
                "postcode": address_params["zipcode"],
                "straatnaam": address_params["street_name"],
                "huisnummer": address_params["house_no"],
            },
        )
        address_match = address_match.json()

        # URL for the building unit
        url_building_unit = address_match["adresMatches"][0]["adresseerbareObjecten"][
            0
        ]["detail"]

        # API request for the building unit
        building_unit = req.get(url_building_unit)
        building_unit = building_unit.json()

        # URL for the building
        url_building = building_unit["gebouw"]["detail"]

        # API request for the building
        building = req.get(url_building)
        building = building.json()

        # Coordinates of the polygon for the building
        building_coords = building["geometriePolygoon"]["polygon"]["coordinates"][0]

        # Polygon for the building
        building_polygon = geom.Polygon(building_coords)

        return building_polygon, building_coords

In [ ]:
class Building(Address):
    """
    A class to represent an building in Flanders
    Inherits from class Address
    ...
    Attributes
    ----------
    input_address : str
        Input address from the user
    metadata_file_path: str (optional)
        File path for the metadata csv file
    valid_address_conditional: bool
        True if the input address is valid
    suggestions: dict
        Suggestions for the input address
    address_info: dict
        Detailed address information, including location
    address: str
        Formatted address
    street_name: str
        Street name 
    house_no: str
        House number
    municipality: str
        Municipality name
    zipcode: str
        Municipality zipcode
    Location: dict
        Location in WGS87 and Lambert73 coordinates
    building_polygon
        Shapely polygon object representing the building
    building_coords
        Coordinates of the building polygon with n sides in the form
        [[x1, y1]
         [x2, y2]
         ...        
         [xn, yn]]
    default_metadata_file_path: str
        Default path for GeoTIFF metadata csv file
        Used if metadata_file_path is not given 
    metadata_all_files
        DataFrame for GeoTIFF metadata for all files
    metadata_single_file
        DataFrame for GeoTIFF file(s) containing the building polygon
        
    Methods
    -------
    get_default_GeoTIFF_metadata_file_path(cls)
        Gets the default file path for the GeoTIFF metadata csv file
    load_GeoTIFF_metadata(cls, csv_file_path: str)
        Loads GeoTIFF metadata from given csv file
    get_GeoTIFF_file_index_single_point(self, x: float, y: float)
        Gets the GeoTIFF file number for the file containing the point (x,y)
    get_building_GeoTIFF_metadata(self)
        Gets the GeoTIFF metadata for the file(s) containing the building polygon
    """

    
    def __init__(self, input_address: str, metadata_file_path: str = ''):
        """
        Initializes an instance of the class Building
        :param input_address: input address from the user
        """
        # 
        super().__init__(input_address)
        
        # For a valid address, get the building information
        if self.valid_address_conditional:
            self.address_info = self.get_address_location(self.input_address)
        else:
            return None
        
        return None
        # Generate default (relative) file path for GeoTIFF metadata
        self.get_default_GeoTIFF_metadata_file_path()
        
        # GeoTIFF Metadata file path 
        if len(metadata_file_path) > 0:
            self.metadata_file_path = metadata_file_path
        else:
            self.metadata_file_path = self.default_metadata_file_path
            
        # Load GeoTIFF metadata
        self.load_GeoTIFF_metadata(self.metadata_file_path)
        
        # Get GeoTIFF file numbers corresponding to the building polygon
        self.get_building_GeoTIFF_metadata()
        
        
    def __str__(self):
        """
        Prints the address
        """
        return f"{self.address}"

    
    @classmethod
    def get_default_GeoTIFF_metadata_file_path(cls):
        """
        Gets the default file path for the GeoTIFF metadata csv file
        """
        # File paths for metadata for DSM and DTM datasets
        metadata_folder_path = "..\\data\\metadata"
        metadata_folder_path = "data\\metadata"
        
        # Metadata csv file path
        default_metadata_file_path = os.path.join(metadata_folder_path, 
                                                       'GeoTIFF_1m_metadata_processed.csv')

        cls.default_metadata_file_path = default_metadata_file_path
    
    @classmethod
    def load_GeoTIFF_metadata(cls, csv_file_path: str):
        """
        Loads GeoTIFF metadata from given csv file
        :param csv_file_path: A string representing an address
        """
        # Load metadata DataFrame
        df_metadata = pd.read_csv(csv_file_path, sep=',', index_col=0)
        
        cls.metadata_all_files = df_metadata
        
    def get_GeoTIFF_file_index_single_point(self, x: float, y: float):
        """
        Gets the GeoTIFF file number for the file containing the point (x,y)
        :param x: x-coordinate of the point
        :param y: y-coordinate of the point
        """
        df = self.metadata_all_files
        index_cond = (x > df.left) & (x <= df.right) & (y > df.bottom) & (y <= df.top)
        index_file = np.flatnonzero(index_cond)[0]

        return index_file

    def get_building_GeoTIFF_metadata(self):
        """
        Gets the GeoTIFF metadata for the file(s) containing the building polygon
        """
        
        # Boundaries of the polygon (left bottom right top)
        polygon_bounds = self.building_polygon.bounds
        
        index_file = []
        # Top left
        index_file.append(self.get_GeoTIFF_file_index_single_point(polygon_bounds[0], polygon_bounds[3]))
        # Top right
        index_file.append(self.get_GeoTIFF_file_index_single_point(polygon_bounds[2], polygon_bounds[3]))
        # Bottom left
        index_file.append(self.get_GeoTIFF_file_index_single_point(polygon_bounds[0], polygon_bounds[1]))
        # Bottom right
        index_file.append(self.get_GeoTIFF_file_index_single_point(polygon_bounds[2], polygon_bounds[1]))
      
        self.metadata_building = (self.metadata_all_files.iloc[index_file]).drop_duplicates()

In [ ]:
nBuffer = 2

raster_file = file_path

dilated_polygon = my_address.building_polygon.buffer(nBuffer)

# Access the TIFF file
try:
    with rs.open(raster_file) as src:
        # Mask the file with the dilated polygon
        masked_raster, masked_transform = rs_mask(src, [dilated_polygon],
                                           all_touched=True, nodata = 0,
                                           filled=True, crop=True,
                                           pad=True, indexes=1)
        
except:
    masked_raster, masked_transform = np.ndarray(0), np.ndarray(0)



In [ ]:
tStart = timeit.default_timer()
print(tStart)
# Do the same for online image
nBuffer = 2

raster_file = "zip+https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k01.zip!/GeoTIFF/DHMVIIDSMRAS1m_k01.tif"

dilated_polygon = my_address.building_polygon.buffer(nBuffer)

try:
    with rs.open(raster_file) as src:
        masked_raster, masked_transform = rs_mask(src, [dilated_polygon],
                                           all_touched=True, nodata = 0,
                                           filled=True, crop=True,
                                           pad=True, indexes=1) 
except Exception as e:
    masked_raster, masked_transform = np.ndarray(0), np.ndarray(0)

tStop = timeit.default_timer()
print(tStop-tStart)
    
     

In [ ]:
# Plot a subset of the layer as a 3D image
layer_slice = my_CHM.masked_raster_chm

# x and y axes
x = range(layer_slice.shape[1])
y = range(layer_slice.shape[0])

# x, y mesh for the surface plot
X, Y = np.meshgrid(x, y)

# Create the figure
hf = plt.figure(figsize=(15,15))
# Add a subplot with 3d projection
ha = hf.add_subplot(111, projection='3d')
# Plot the surface
ha.plot_surface(X, Y, layer_slice)
plt.xlabel('x')
plt.ylabel('y')
ha.invert_xaxis()
ha.invert_yaxis()
plt.show()
